# ETL com Google Sheets + OpenAI


In [ ]:
!pip install gspread oauth2client openai

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
sheet_id = "COLE_AQUI_O_ID_DA_SUA_PLANILHA"

sheet = gc.open_by_key(sheet_id).sheet1
user_ids = sheet.col_values(1)[1:]
user_ids = [int(x) for x in user_ids]

print("User IDs encontrados:", user_ids)

In [ ]:
from openai import OpenAI
openai_api_key = "COLE_AQUI_SUA_API_KEY"
client = OpenAI(api_key=openai_api_key)

In [ ]:
def get_user_simulado(id):
    nomes = ["Ana", "Bruno", "Carla", "Diego", "Eduarda"]
    return {
        "id": id,
        "name": nomes[(id - 1) % len(nomes)],
        "account": {"balance": round(1000 * id * 0.77, 2)}
    }

users = [get_user_simulado(id) for id in user_ids]
users

In [ ]:
def gerar_mensagem(user):
    prompt = (
        f"Crie uma mensagem curta (máx 100 caracteres) para {user['name']} sobre a importância dos investimentos."
    )

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Você é um especialista em marketing bancário."},
            {"role": "user", "content": prompt}
        ]
    )

    return completion.choices[0].message.content

In [ ]:
for u in users:
    u["news"] = gerar_mensagem(u)
    print(u["name"], "=>", u["news"])

In [ ]:
sheet.update("B1", "Name")
sheet.update("C1", "News")

for idx, user in enumerate(users, start=2):
    sheet.update(f"B{idx}", user["name"])
    sheet.update(f"C{idx}", user["news"])

print("Atualização concluída no Google Sheets!")